In [1]:
clean_up=True # removes gams-related files in work-folder if true
%run StdPackages.ipynb
os.chdir(directory['py'])
import abatement, techdata_to_tree, sys, ShockFunction
os.chdir(directory['curr'])
def flatten_list(list_):
    return [item for sublist in list_ for item in sublist]

The file_gams_py_gdb0.gdx is still active and was not deleted.


## 1: Set up tree

*Read data:*

In [2]:
inputfile = "techdata_dors_2.xlsx" # file
inputoth  = 'Othdata_dors_2.xlsx'
output = techdata_to_tree.load_techcats(pd.read_excel(directory['data'] + "/" + inputfile, sheet_name=["inputdisp", "endofpipe", "inputprices"]))
modules = ["ID", "EOP"] # modules to use

*Set up trees:*

In [3]:
nts = {m: nesting_tree.nesting_tree(name=m) for m in modules}

#### 1.1: Input displacing tree

In [4]:
nts["ID"].add_tree(output["ID"]["upper_categories"], tree_name = 'ID_EC', **{"type_f":"CES_norm"}) # E to C
nts["ID"].add_tree(output["ID"]["components"], tree_name = "ID_CU", **{"type_f":"MNL"}) # C to U
nts["ID"].add_tree(output["ID"]["techs"], tree_name="ID_TU", **{'type_io': 'output', 'type_f': 'CET_norm'}) # U to T
nts["ID"].add_tree(output["ID"]["techs_inputs"], tree_name="ID_TX") # T to inputs X
nts["ID"].add_tree(output["ID"]["basetechs"], tree_name="ID_BU", **{"type_io":"output", "type_f":"linear_out"}) # baseline technologies
nts["ID"].add_tree(output["ID"]["basetech_inputs"], tree_name="ID_BX")

#### 1.2: End of pipe tree

In [5]:
nts["EOP"].add_tree(output["EOP"]["components"], tree_name = "EOP_CU", **{"type_f":"MNL"}) # C to U
nts["EOP"].add_tree(output["EOP"]["techs"], tree_name="EOP_TU", **{'type_io': 'output', 'type_f': 'CET_norm'}) # T to U 
nts["EOP"].add_tree(output["EOP"]["techs_inputs"], tree_name="EOP_TX") # T to inputs X

#### 1.3: Trees related to final goods and emissions account

*Read in tree and data from xlsx:*

In [6]:
nts['ID'].add_tree(directory['data']+'\\'+inputoth,tree_name='ID_Y',**{'sheet':'Y'})
DataBase.GPM_database.merge_dbs(nts['ID'].trees['ID_Y'].database,excel2py.xl2PM.pm_from_workbook(directory['data']+'\\'+inputoth,{'Y': 'vars'}), 'first')

*Add new Q2P:*

In [7]:
output['ID']['Q2P'] = output['ID']['Q2P'].append(excel2py.xl2PM.pm_from_workbook(directory['data']+'\\'+inputoth,{'Q2P': 'maps'}).database['Q2P'].vals)

#### 1.3: Namespaces

*Give modules local namespaces:*

In [8]:
standard_sets = ('inp','out','int','wT','map_all','kno_out','kno_inp')
for module in modules:
    namespace = {k: module + '_' + k for k in standard_sets}
    nts[module].run_all(**namespace)
    #Also replaces keys with module-specific names, i.e. changes 'inp' to 'ID_inp' in the attributes/keys.
    for std_set in standard_sets:
        if hasattr(nts[module], std_set):
            setattr(nts[module], module + "_" + std_set, getattr(nts[module], std_set))
            delattr(nts[module], std_set)

*Add emissions data:*

In [9]:
DataBase.GPM_database.merge_dbs(nts['ID'].database,excel2py.xl2PM.pm_from_workbook(directory['data']+'\\'+inputoth, {'M': 'vars','M_sets': 'sets'}),'first')

## 2: Initialize model

*Initialize:*

In [10]:
gm = abatement.abate(nt=nts['ID'],tech=output,work_folder=directory['work'],**{'data_folder':directory['gams'],'name': 'atest'})

*Add a couple of extra aliases, and update sets with elements:*

In [11]:
gm.add_aliases([('n','nnnn'),('n','nnnnn')])
gm.model.database.update_all_sets(clean_up=False)

*Debugging of initial states:*

In [12]:
debug = False
if debug is True:
    gm.initialize_variables(**{'check_variables': True})
    from IPython.display import Markdown
    # Test groupings:*
    var = gm.default_variables[1]
    display(Markdown(f"**{var}+' exogenous:'**"))
    display(gm.var_exo(var))
    display(Markdown(f"**{var}+' endogenous:'**"))
    display(gm.var_endo(var))

*Debugging of writing groups:*

In [13]:
group = gm.model.settings.name+'_'+'g_ID_alwaysendo'
debug_group = False
if debug_group is True:
    gm.initialize_variables(**{'check_variables':True})
    if group in gm.exo_groups:
        group_vals = gm.exo_groups[group] # corresponding group conditions 
    else:
        group_vals = gm.endo_groups[group]
    gm.model.groups[group] = group_vals # add to model.groups
    gm.model.settings.get_conf('g_exo').update(group) # add to exogenous groups
    repo = None
    self = gm.model
    if repo is None:
        repo = self.settings.data_folder
    if not os.path.exists(repo):
        os.makedirs(repo)
    self.write_default_components()
    self.default_export(repo)

In [14]:
gm.write_and_run(kwargs_init={'check_variables':True})

GamsExceptionExecution: GAMS return code not 0 (3), check C:\Users\sxj477\Documents\GitHub\GPM_v05\work_folder\_gams_py_gjo0.lst for more details

### ID sums:

* Define qSumU[n,nn] as the production by technology (nn) of energy service (n). This should be computed as: 

$$
\begin{align}
    \text{qSumU[n,nn]} = \text{sum(nnn\$(e2t[n,nn] and e2u[n,nnn]), qD[nnn])}.
\end{align}
$$

* Define the output share os[n,nn] as the share of output that technology $nn$ produces of energy-service type $n$. This should be defined as:

$$
\begin{align}
    \text{os[n,nn]} = \text{sum(nnn\$(e2u[n,nnn] and u2t[nnn,nn]), qD[nnn]) / qD[nn]}.
\end{align}
$$

* Define variable qSumX[n,nn] defined as the use of aggregate input type $nn\in (electricity,oil,inp3,K)$ used to produce energy service $n$. This is defined as:

$$\begin{align}
    \text{qSumX[n,nn]} = \text{sum([nnn,nnnn]\$(e2ai2i[n,nn,nnn] and e2t[n,nnnn] and i2t[nnn,nnnn]), qD[nnn] * os[n,nnnn])}
\end{align}$$
* Here e2ai2i identifies the relevant quantities $qD[i]$ that are tied to relevant aggregate type of input, $e2t[e,t]$ identifies the correct output-share $os[t,e]$, and $i2t[i,t]$ ensures that only the relevant $i,t$ combinations are summed over.


### ID emissions:

Emissions accounting with only the input displacing module:
* $M0[z]$ defines pre-emission abatement (equivalent to emissions without EOP module). This is defined over z (emission types).
* $phi[z,n]$ defines mission intensities over aggregate input types ($n\in ai[n]$).
Pre abatement emissions are then defined as:

$$\begin{align}
    M0[z] = sum(n$(ai[n]) phi[z,n] * qD[n]), 
\end{align}$$
* As long as the equation for the aggregate use of inputs in the sector (qD[n] for ai[n]) takes into account which modules are active, this equation stays the same across all states.

* $PwThat[n]\$(ID\_inp[n]):$ defines the effective prices on inputs, taking the price on emissions taxes and abatement into account:

$$\begin{align}
    PwThat[n] = PwT[n] + sum(z, phi[z,n] * pMhat[z]);
\end{align}$$

* As long as we establish an equation for pMhat[z] (corrected emission prices) that depends on the type of state the model is in, this equation stays the same across all states. However, with the EOP module activated, there should be an equivalent equation defined for ($EOP\_inp[n]$ and not $ID\_inp[n]$).

### ID Aggregates:

* If the state is ID the aggregate use of inputs ($qD[n]\$(ai[n])$) are defined from (we could let the namespace for 'ai' look up 'ID\_ai' instead of 'ai' when the EOP set is not activated?)

$$\begin{align}
    qD[n] = sum(nn\$(ID\_i2ai[nn,n]), qD[nn])
\end{align}$$

* Without the EOP module, a second equation simply states that $pMhat = pM$. 